##### 0. Notebook setup and imports <br>
Here we make the preparation necessary for the notebook to run

In [6]:
# 0.1 Notebook setup

import sys
import os
print(sys.path)

NOTEBOOK_DIR = os.getcwd()
print(NOTEBOOK_DIR)
ROOT_DIR = os.path.dirname(NOTEBOOK_DIR)
print(ROOT_DIR)
SRC_DIR = os.path.join(ROOT_DIR, 'src')
print(SRC_DIR)

# Adds project's roof to sys.path
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\win32', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\Claudia\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Pyth

In [7]:
# 0.2 Imports

import pandas as pd
import sqlite3
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
from indicators import calculate_sma, calculate_returns_std, calculate_atr, calculate_bollinger_bands

#### 1. Querying data for dataviz <br>
##### Now we're querying the SQL table with bitcoin indicators to save it in a DataFrame and create our visualization.

In [8]:
# Querying 11:00 PM close dates

connection = sqlite3.connect('../data/crypto.db') #connects Python to SQLite
cursor = connection.cursor()


query = ("""
        SELECT
            date,
            open,
            high,
            low,
            close,
            volume
        FROM bitcoin_daily_indicators
        ORDER BY date;
""")

bitcoin_daily_df = pd.read_sql_query(query, connection, parse_dates=['date'], index_col='date')
connection.close()

print(bitcoin_daily_df.columns)

bitcoin_daily_df 


Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')


,open,high,low,close,volume
date,,,,,
2025-01-06,102047.455421,102383.400987,101640.113923,102280.873055,6498.484159
2025-01-07,97182.054801,102713.731333,96177.422448,96974.635009,11037.850832
2025-01-08,95301.165656,97292.115715,92645.249449,95087.271746,6860.542590
2025-01-09,92521.972997,95376.246571,91266.677130,92580.981277,12987.622473
2025-01-10,94783.697336,95784.332869,92324.819641,94735.725063,5015.988445
...,...,...,...,...,...
2025-04-25,95070.540212,95778.630924,92897.633538,95053.349524,4586.891899
2025-04-26,94145.758490,95338.083177,93932.787277,94382.522065,4270.546077
2025-04-27,93015.148977,94566.721416,92881.602296,93681.223198,8883.259669


#### 2. Dataviz <br>
##### Using Plotly, we will create interactive data with custom window dates for our analysis.  
##### 7, 14, 50 and 100 days were the chosen windows, but the 'window_options' list allow us to create customized windows if the developer need to.

In [10]:
# Plotting the data

def plot_interactive_indicators(df):
    """Plots Bitcoin price in USD, simple moving average (SMA), and ATR (Average True Range) accordingly with the user's adjustment
    Args:
        df (pd.Dataframe): DataFrame with OHLC data and datetime index"""
    
    # Defining dropdown menus
    window_options = [7, 14, 30, 50, 100]      # <---- YOU CAN CUSTOMIZE THE DAYS HERE
        # SMA Dropdown
    sma_dropdown = widgets.Dropdown(
        options=window_options,
        value=14,
        description='SMA Window'
    )

        # ATR Dropdown
    atr_dropdown = widgets.Dropdown(
        options=window_options,
        value=14,
        description='ATR Window'
    )

        
        #STD Dropdown
    std_dropdown=widgets.Dropdown(
        options=window_options,
        value=14,
        description='STD Dropdown'
    )


    def update_plot(sma_window, atr_window, std_window):
        fig = make_subplots(
            rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.04,
            row_heights=[0.25, 0.1, 0.1, 0.1, 0.3],
            subplot_titles=('<i>Closing Price with Simple Moving Average and Bollinger Bands</i>',
                            f'<i>Daily Average True Range (ATR)</i>',
                            '<i>Volatility (line plot)</i>',
                            '<i>Volatility (histogram)</i>',
                            '<i>Candlestick Chart</i>')
        )


        # SMA and Bollinger Bands
            # Calculating SMA
        sma_column_name = f'SMA_{sma_window}d'
        if sma_column_name in df.columns:
            sma_data = df[sma_column_name]
        else:
            sma_data = calculate_sma(df['close'], sma_window)
            # Calculating Upper and Lowers Bands
        upper_band_column_name = f'BB_Upper_{sma_window}d'
        lower_band_column_name = f'BB_Lower_{sma_window}d'
        if f'BB_Upper_{sma_window}d' in df.columns and f'BB_Lower_{sma_window}d':
            upper_band_data = df[upper_band_column_name]
            lower_band_data = df[lower_band_column_name]
        else:
            upper_band_data, lower_band_data = calculate_bollinger_bands(df['close'], sma_window, 2)
            # Plotting Upper Band
        fig.add_trace(go.Scatter(x=df.index, y=upper_band_data, name='Upper Band',
                                 line=dict(color='#75942b'),
                                 hovertemplate='Upper Band: %{y:.1f} USD<extra></extra>'), row=1, col=1)
            # Plotting Lower Band
        fig.add_trace(go.Scatter(x=df.index, y=lower_band_data, name='Lower Band',
                                 line=dict(color='#75942b'),
                                 hovertemplate='Lower Band: %{y:.1f} USD<extra></extra>',
                                 fill='tonexty', fillcolor='rgba(117,148,43,0.2)'))
        
        # Plotting closing price
        fig.add_trace(go.Scatter(x=df.index, y=df['close'], name='Closing price',
                                 line=dict(color='#195EFB'), hovertemplate='Closing price: %{y:.1f} USD<extra></extra>'), row=1, col=1)
        
        # Plotting SMA
        fig.add_trace(go.Scatter(x=df.index, y=sma_data,
                                 name=f'Moving Average ({sma_window} days)',
                                 line=dict(color='#ec4a39'), hovertemplate='SMA: %{y:.1f} USD<extra></extra>'), row=1, col=1)
        fig.update_yaxes(
        title='Closing Price (USD)',
        row=1,
        col=1
        )

        # ATR
        atr_column_name = f'ATR_{atr_window}d'
        if atr_column_name in df.columns:
            atr_data = df[atr_column_name]
        else:
            atr_data = calculate_atr(df[['high', 'low', 'close']], atr_window)
            # Plotting ATR
        fig.add_trace(go.Scatter(x=df.index, y=atr_data,
                                 name=f'ATR {atr_window} day(s)',
                                 line=dict(color='#ddaf46'), hovertemplate='ATR: %{y:.1f}<extra></extra>'), row=2, col=1)
        
        # STD
        std_column_name = f'Return_Std_{std_window}d'
        if std_column_name in df.columns:
           std_data = df[std_column_name]
        else:
            std_data = calculate_returns_std(df['close'], std_window)
            #Plotting STD
        fig.add_trace(go.Scatter(x=df.index, y=std_data,
                                  name=f'Standard Deviation of the Returns ({std_window} days)',
                                  line=dict(color='pink'), hovertemplate='STD: %{y:.2f}% <extra></extra>'), row=3, col=1)
        fig.add_trace(go.Histogram(x=std_data, marker_line_color='#f2b5e2', marker_line_width=1.5, nbinsx=10, name='Standard deviation of returns'), row=4, col=1)
        fig.update_yaxes(title_text='Frequency', row=4, col=1)

        fig.update_layout(title_text="<b>Bitcoin Interactive Daily Indicators</b>",  title_x=0.5, showlegend=True, height=2000, width=1750,
                        title_font=dict(size=18),
                        font_color='rgb(226,212,221)',
                        plot_bgcolor='#1A191A',
                        paper_bgcolor='rgb(36,34,35)',
                        hovermode='x unified',
                        bargap=0.05)       
        
        for axis_name, axis_obj in fig.to_plotly_json()['layout'].items():
            if axis_name.startswith('xaxis') or axis_name.startswith('yaxis'):
                fig.update_layout({axis_name: dict(showgrid=True, gridcolor='rgba(140,148,148,0.1)')})
   
        fig.update_yaxes(
        tickformat=".2f%",
        title='Std of returns (%)',
        row=3,
        col=1
        )

        # Candlestick
        fig.add_trace(go.Candlestick(x=df.index,
                                     open=df['open'],
                                     high=df['high'],
                                     low=df['low'],
                                     close=df['close'],
                                     name='Price'), row=5, col=1)
        
        # Volume
        volume_colors = ['green' if c >= o else 'red' for c, o in zip(df['close'], df['open'])]
        fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume',
                            marker=dict(color=volume_colors,
                            opacity=0.4,
                            line=dict(width=0))), row=5, col=1)
        fig.update_yaxes(title_text='Volume', row=5, col=1)

        fig.show()

    ui = widgets.HBox([sma_dropdown, atr_dropdown, std_dropdown])
    out = widgets.interactive_output(update_plot, {'sma_window': sma_dropdown, 'atr_window': atr_dropdown, 'std_window': std_dropdown})
    
    display(ui, out)


connection = sqlite3.connect('../data/crypto.db') #connects Python to SQLite
cursor = connection.cursor()

query_daily_indicators = ("""
    SELECT *
    FROM bitcoin_daily_indicators
    ORDER BY date;
""")
    
bitcoin_daily_df = pd.read_sql_query(query_daily_indicators, connection, parse_dates=['date'], index_col='date')
connection.close()

   
plot_interactive_indicators(bitcoin_daily_df.copy())

Output()